In [1]:
import sys
sys.path.append("../..")
import torch
from draw import make_html
import py3Dmol
import numpy as np
from metrics import check_validity_smiles

samples = torch.load("../../samples/gen/ds_qm9_original_rep0/gen.pt", weights_only=False)
res = 560

for chunk in range(4):
    view = py3Dmol.view(width=res*4, height=res*5, viewergrid=(5, 4))
    for i in range(20):
        b_idx = chunk * 20 + i + 160
        row = i % 5
        col = i // 5

        M = samples[0][0][b_idx].clone()
        for i, a in enumerate(M.atoms):
            if a == 0:
                break
        M.atoms = M.atoms[:i]
        M.coords = M.coords[:i]
        M.coords = M.coords - M.coords.mean(dim=0, keepdim=True)

        # PCA for visualization
        U, _, _ = np.linalg.svd(M.coords.T.numpy())
        if np.linalg.det(U) < 0:
            U[:, -1] *= -1
        U = torch.tensor(U)
        M.coords = M.coords @ U

        view = M.show(view=view, viewer=(row, col))

        yes, _ = check_validity_smiles(M)
        if yes:
            bg = "#cffbcf"
        else:
            bg = "#f4ccca"
        view.setBackgroundColor(bg, viewer=(row, col))
    
    # view.show()
    path = f"qm9_chunk{chunk}.html"
    make_html(view, path)

In [2]:
samples = torch.load("../../samples/gen/ds_geom_original_test201_rep0/gen.pt", weights_only=False)
res = 560

for chunk in range(4):
    view = py3Dmol.view(width=res*4, height=res*5, viewergrid=(5, 4))
    for i in range(20):
        b_idx = chunk * 20 + i
        row = i % 5
        col = i // 5

        M = samples[0][0][b_idx].clone()
        for i, a in enumerate(M.atoms):
            if a == 0:
                break
        M.atoms = M.atoms[:i]
        M.coords = M.coords[:i]
        M.coords = M.coords - M.coords.mean(dim=0, keepdim=True)

        # PCA for visualization
        U, _, _ = np.linalg.svd(M.coords.T.numpy())
        if np.linalg.det(U) < 0:
            U[:, -1] *= -1
        U = torch.tensor(U)
        M.coords = M.coords @ U

        view = M.show(view=view, viewer=(row, col))

        yes, _ = check_validity_smiles(M)
        if yes:
            bg = "#cffbcf"
        else:
            bg = "#f4ccca"
        view.setBackgroundColor(bg, viewer=(row, col))
    
    view.zoom(0.94)
    # view.show()
    path = f"geom_chunk{chunk}.html"
    make_html(view, path)